# 모델링

In [1]:
import pandas as pd

# 데이터셋 로드
train = pd.read_csv('전기차_가격_예측train.csv')
test = pd.read_csv('전기차_가격_예측test.csv')

# 데이터 전처리
# 'train'과 'test' 데이터에서 'ID' 열을 제거하고, 'test'의 'ID'는 별도로 저장
train = train.drop('ID', axis = 1)
ID = test.pop('ID')

# 의미 없는 변수 제거
train = train.drop(['구동방식', '차량상태', '사고이력'], axis = 1)
test = test.drop(['구동방식', '차량상태', '사고이력'], axis = 1)

# 결측값 처리: 결측값을 '배터리용량' 열의 평균값으로 채움
m = train['배터리용량'].mean()
train = train.fillna(m)
test = test.fillna(m)

# 타겟 변수 저장
target = train.pop('가격(백만원)')

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.columns[train.dtypes == object]

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

# 모델 학습
# XGBoost
import xgboost as xgb
xg = xgb.XGBRegressor(max_depth = 7, n_estimators = 1870, learning_rate = 0.02, random_state = 0)
xg.fit(train, target)

# 제출
pred = xg.predict(test)
submit = pd.DataFrame({'ID':ID,
                    '가격(백만원)':pred})
submit.to_csv('전기차_가격_예측sample_submission.csv', index = False)